# Data Acquisition

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [15]:
base_URL = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina='

In [16]:
# get ads in multiple pages and returns a list of dataFrames
def scrap(url, pages=2):
    '''Scraps search results from a real state website'''
    if pages < 2:
        raise ValueError('number of pages must be greater than 2')
    scrap_df = [] # list to hold all data frames created per page
    for p_no in range(1, pages):
        search_url = url + str(p_no)
        # check if request is successful
        try:
            page_content = requests.get(search_url).content
            print(search_url)
        except Exception as e:
            print(f'Failed to gather data from {search_url}')
            print(e)
    
        soup = BeautifulSoup(page_content) # get page content into soup

        rent_tag_list = soup.find_all('div', {'class': 'property-card__price js-property-card-prices js-property-card__price-small'}) # gets all rent tags
        rent_text_list = [rent.p.text for rent in rent_tag_list] # gets the text from each rent tag


        rooms_tag_list = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-room js-property-detail-rooms'})
        rooms_text_list = [room.span.text for room in rooms_tag_list]


        address_tag_list = soup.find_all('span', {'class': 'property-card__address'})
        address_text_list = [address.text for address in address_tag_list]


        bathroom_tag_list = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'})
        bathroom_text_list = [bathroom.span.text for bathroom in bathroom_tag_list]


        parking_tag_list = soup.find_all('li', {'class': 'property-card__detail-item property-card__detail-garage js-property-detail-garages'})
        parking_text_list = [parking.span.text for parking in parking_tag_list]

        area_tag_list = soup.find_all('span', {'class': 'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area'})
        area_text_list = [area.text for area in area_tag_list]

        # create a dictionary to create a pandas dataframe
        data = {'address': address_text_list,
                'rent': rent_text_list,
                'rooms': rooms_text_list,
                'bathroom': bathroom_text_list,
                'parking': parking_text_list,
                'area': area_text_list}
        
        #create the dataFrame
        df = pd.DataFrame(data)
        print(df)
        scrap_df.append(df)
    
    final_data = pd.concat(scrap_df)
    # return final dataFrame
    return final_data

In [17]:
final_data = scrap(base_URL, 180)

, São Paul...    R$ 2.700 /Mês     1    
9   Avenida Onze de Junho, 730 - Vila Clementino, ...    R$ 2.290 /Mês     2    
10      Rua Helena, 52 - Vila Olímpia, São Paulo - SP    R$ 4.200 /Mês     2    
11  Rua Marquês de Itu, 181 - Vila Buarque, São Pa...    R$ 2.500 /Mês     2    
12  Rua Domingos da Costa Mata, 275 - Santana, São...    R$ 1.700 /Mês     3    
13  Rua Henri Dunant, 1066 - Santo Amaro, São Paul...    R$ 2.880 /Mês     1    
14  Rua Volta Redonda, 757 - Campo Belo, São Paulo...    R$ 7.000 /Mês     2    
15  Rua Padre Carvalho, 68 - Pinheiros, São Paulo ...    R$ 2.950 /Mês     2    
16  Rua Doutor Seng, 152 - Bela Vista, São Paulo - SP    R$ 1.500 /Mês     1    
17  Rua Judith Zumkeller, 870 - Parque Mandaqui, S...    R$ 2.500 /Mês     2    
18                 Vila Santo Estevão, São Paulo - SP    R$ 3.500 /Mês     4    
19  Rua João Lourenço, 763 - Vila Nova Conceição, ...    R$ 5.500 /Mês     3    
20  Rua Padre Bento Ibañez - Jardim Prudência, São...    R$ 1.300 /M

In [18]:
final_data.head(72)

,address,rent,rooms,bathroom,parking,area
0,"Rua Oscar Freire, 1735 - Pinheiros, São Paulo ...",R$ 3.400 /Mês,2,2,1,80
1,"Rua Celso Ramos, 145 - Vila Andrade, São Paulo...",R$ 1.950 /Mês,2,1,1,47
2,"Praça da República, 401 - República, São Paulo...",R$ 2.300 /Mês,1,1,1,30
3,"Rua dos Caciques - Vila da Saúde, São Paulo - SP",R$ 1.800 /Mês,1,2,1,37
4,"Rua das Perobeiras - Chácara Santa Maria, São ...",R$ 900 /Mês,2,1,1,45
...,...,...,...,...,...,...
31,"Rua Bacaetava, 66 - Vila Gertrudes, São Paulo ...",R$ 6.000 /Mês,3,3,2,97
32,"Rua Verbo Divino, 1061 - Chácara Santo Antônio...",R$ 9.500 /Mês,3,5,4,186
33,"Pinheiros, São Paulo - SP",R$ 1.900 /Mês,1,1,--,25
34,"Rua Guarani, 460 - Bom Retiro, São Paulo - SP",R$ 1.600 /Mês,2,1,--,60


In [19]:
final_data['suburb'] = final_data.address.str.split('\s-').str[1].str.split(', ').str[0]

In [20]:
final_data

,address,rent,rooms,bathroom,parking,area,suburb
0,"Rua Oscar Freire, 1735 - Pinheiros, São Paulo ...",R$ 3.400 /Mês,2,2,1,80,Pinheiros
1,"Rua Celso Ramos, 145 - Vila Andrade, São Paulo...",R$ 1.950 /Mês,2,1,1,47,Vila Andrade
2,"Praça da República, 401 - República, São Paulo...",R$ 2.300 /Mês,1,1,1,30,República
3,"Rua dos Caciques - Vila da Saúde, São Paulo - SP",R$ 1.800 /Mês,1,2,1,37,Vila da Saúde
4,"Rua das Perobeiras - Chácara Santa Maria, São ...",R$ 900 /Mês,2,1,1,45,Chácara Santa Maria
...,...,...,...,...,...,...,...
31,"Avenida Paulista, 37 - Bela Vista, São Paulo - SP",R$ 3.350 /Mês,3,2,1,116,Bela Vista
32,"Rua Conselheiro Moreira de Barros, 884 - Santa...",R$ 3.000 /Mês,3,4,3,156,Santana
33,"Rua Lisboa, 159 - Cerqueira César, São Paulo - SP",R$ 5.380 /Mês,2,2,1,163,Cerqueira César
34,"Rua Pereira do Lago, 309 - Vila Gomes, São Pau...",R$ 990 /Mês,1,1,--,20,Vila Gomes


In [21]:
pd.DataFrame.nunique(final_data)

address     82
rent        52
rooms        4
bathroom     6
parking      6
area        57
suburb      48
dtype: int64

In [22]:
pd.DataFrame.drop_duplicates(final_data)

,address,rent,rooms,bathroom,parking,area,suburb
0,"Rua Oscar Freire, 1735 - Pinheiros, São Paulo ...",R$ 3.400 /Mês,2,2,1,80,Pinheiros
1,"Rua Celso Ramos, 145 - Vila Andrade, São Paulo...",R$ 1.950 /Mês,2,1,1,47,Vila Andrade
2,"Praça da República, 401 - República, São Paulo...",R$ 2.300 /Mês,1,1,1,30,República
3,"Rua dos Caciques - Vila da Saúde, São Paulo - SP",R$ 1.800 /Mês,1,2,1,37,Vila da Saúde
4,"Rua das Perobeiras - Chácara Santa Maria, São ...",R$ 900 /Mês,2,1,1,45,Chácara Santa Maria
...,...,...,...,...,...,...,...
31,"Estrada de Itapecerica, 1528 - Vila Prel, São ...",R$ 1.900 /Mês,2,1,1,48,Vila Prel
32,"Rua Peixoto Gomide - Jardim Paulista, São Paul...",R$ 3.900 /Mês,2,1,1,52,Jardim Paulista
33,"Rua Vupabussu - Pinheiros, São Paulo - SP",R$ 3.200 /Mês,2,1,1,56,Pinheiros
34,"Rua Plínio Salgado, 245 - Jardim Peri Peri, Sã...",R$ 850 /Mês,1,1,--,15,Jardim Peri Peri
